In [190]:
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact
plt.rcParams['figure.dpi'] = 200

In [191]:
file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [192]:
x_list = []
y_list = []

# The bigger the time step, the more the history. 
TIME_STEPS = 64
SAMPLE_STEPS = 4

x_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]
y_labels = ["a0", "a1", "a2", "a3", "a4", "a5", "d"]

INPUT_DIM = len(x_labels)
OUTPUT_DIM = len(y_labels)

def get_index_groups(df):
    return poly_df.groupby(poly_df.index.to_series().diff().ne(1).cumsum()).groups

def expand_time_steps(data, time_steps, sample_steps):
    entries = data.shape[0] - time_steps + 1
    rows = int(time_steps / sample_steps)
    cols = data.shape[1]
    new_arr = np.zeros((entries, rows, cols))
    for i in range(data.shape[0] - time_steps + 1):
        full_arr = data.iloc[i:i + time_steps].to_numpy()
        new_arr[i, :, :] = full_arr[sample_steps - 1::sample_steps]
    return new_arr

def normalize_2d(data):
    min_vals = []
    max_vals = []
    rows = data.shape[0]
    cols = data.shape[1]
    for c in range(cols): 
        mx = data[0][c]
        mn = mx
        for r in range(rows):
            val = data[r][c]
            if val > mx:
                mx = val
            if val < mn:
                mn = val
        min_vals.append(mn)
        max_vals.append(mx)
        for r in range(rows):
            val = data[r][c]
            data[r][c] = (val - mn) / (mx - mn)
    return min_vals, max_vals
    
def normalize_3d(data):
    min_vals = []
    max_vals = []
    entries = data.shape[0]
    rows = data.shape[1]
    cols = data.shape[2]
    for c in range(cols):
        mx = data[0][0][c]
        mn = mx
        for e in range(entries):
            for r in range(rows):
                val = data[e][r][c]
                if val > mx:
                    mx = val
                if val < mn:
                    mn = val
        min_vals.append(mn)
        max_vals.append(mx)
        for e in range(entries):
            for r in range(rows):
                val = data[e][r][c]
                data[e][r][c] = (val - mn) / (mx - mn)
    return min_vals, max_vals

for header in file_headers:
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    poly_df = pd.read_csv(poly_file)
    
    # NOTE: Handle when the robot goes out of frame.
    zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
    
    data_df = data_df.drop(data_df.index[zero_indices])
    poly_df = poly_df.drop(poly_df.index[zero_indices])
    
    data_groups = get_index_groups(data_df)
    poly_groups = get_index_groups(poly_df)
    
    for dg in data_groups:
        idx_list = data_groups[dg]
        sub_data_df = data_df.loc[idx_list[0]:idx_list[-1]+1, x_labels]
        sub_poly_df = poly_df.loc[idx_list[0]:idx_list[-1]+1, y_labels]
        
        sub_data_df_exp = expand_time_steps(sub_data_df, TIME_STEPS, SAMPLE_STEPS)
        sub_poly_df_exp = sub_poly_df[TIME_STEPS-1:]
        
        print(sub_data_df_exp.shape)
        print(sub_poly_df_exp.shape)
        
        x_list.append(sub_data_df_exp)
        y_list.append(sub_poly_df_exp.to_numpy())
        
x_data = np.concatenate(x_list, axis=0)
y_data = np.concatenate(y_list, axis=0)

x_mins, x_maxes = normalize_3d(x_data)
y_mins, y_maxes = normalize_2d(y_data)

x_data_shuffled, y_data_shuffled = sklearn.utils.shuffle(x_data, y_data)

data_length = len(x_data_shuffled)
split_percent = 0.8
train_test_split = int(data_length * split_percent)

x_train = x_data_shuffled[:train_test_split]
y_train = y_data_shuffled[:train_test_split]

x_test = x_data_shuffled[train_test_split:]
y_test = y_data_shuffled[train_test_split:]

INPUT_TIME_STEPS = x_train.shape[1]


(418, 16, 4)
(418, 7)
(443, 16, 4)
(443, 7)
(448, 16, 4)
(448, 7)
(555, 16, 4)
(555, 7)
(537, 16, 4)
(537, 7)
(463, 16, 4)
(463, 7)
(377, 16, 4)
(377, 7)
(394, 16, 4)
(394, 7)
(221, 16, 4)
(221, 7)
(645, 16, 4)
(645, 7)
(186, 16, 4)
(186, 7)
(676, 16, 4)
(676, 7)


In [193]:
model = Sequential()
#model.add(LSTM(64, input_shape=(INPUT_TIME_STEPS, INPUT_DIM), activation='relu', return_sequences=True))
model.add(LSTM(64, input_shape=(INPUT_TIME_STEPS, INPUT_DIM), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(OUTPUT_DIM, activation='relu'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 64)                17664     
_________________________________________________________________
dropout_53 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_54 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_56 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_55 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 16)              

In [200]:
BATCH_SIZE = 8
EPOCHS = 50
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0032 - accuracy: 0.8149
Epoch 2/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0032 - accuracy: 0.8231
Epoch 3/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0031 - accuracy: 0.8212
Epoch 4/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0031 - accuracy: 0.8191
Epoch 5/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0032 - accuracy: 0.8159
Epoch 6/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0032 - accuracy: 0.8121
Epoch 7/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0030 - accuracy: 0.8184
Epoch 8/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0032 - accuracy: 0.8140
Epoch 9/50
537/537 [==============================] - 1s 3ms/step - loss: 0.0032 - accuracy: 0.8212
Epoch 10/50
537/537 [==============================] - 2s 3ms/step - loss: 0.0033 - accuracy: 0.8275

In [201]:
loss, accuracy = model.evaluate(x_test, y_test)

34/34 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 0.8686


In [198]:
def rescale_output(y):
    ret = []
    for i in range(len(y)):
        max_val = y_maxes[i]
        min_val = y_mins[i]
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [202]:
num_tests = len(x_test)
def evaluate_poly(idx=(0,num_tests-1,1)):
    x_select = np.expand_dims(x_test[idx], axis=0)
    #x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    a_pred = np.flip(y_pred[:-1])
    d_pred = y_pred[-1]
    
    y_select = y_test[idx]
    y_select = rescale_output(y_select)
    a_select = np.flip(y_select[:-1])
    d_select = y_select[-1]
    
    print(d_pred, d_select)
    
    poly_pred = np.poly1d(a_pred)
    poly_select = np.poly1d(a_select)
    
    yp = np.linspace(0, d_pred)
    xp = poly_pred(yp)
    
    ys = np.linspace(0, d_select)
    xs = poly_select(ys)
    
    plt.plot(xs, ys)
    plt.plot(xp, yp)
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.show()
    

interact(evaluate_poly)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_poly(idx=(0, 1072, 1))>

In [145]:
a = np.array([1,2,3,4,5,6,7,8])
a[3::4]

array([4, 8])

In [173]:
model.save("./models/first_rnn")

INFO:tensorflow:Assets written to: ./models/first_rnn/assets
